In [1]:
text = "hello I am happy. Are you?"

In [2]:
# libraries for: regular expressions, file I/O
import re
import sys
import unicodecsv

In [3]:
def preProcess(text):
    #print "original:", text
    # sentence segmentation - assume already done
    pass
    # word tokenisation
    text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", text)
    text = re.sub(r"([.,;:!?'\"“\(])(\w)", r"\1 \2", text)
    #print "tokenising:", text
    tokens = re.split(r"\s+",text)
    # normalisation
    text = re.sub(r"(\S)\1\1+",r"\1\1\1", text)
    tokens = [t.lower() for t in tokens]
    return tokens

In [4]:
print preProcess(text)

['hello', 'i', 'am', 'happy', '.', 'are', 'you', '?']


In [5]:
# turn into feature vectors
featureDict = {}
def toFeatureVector(words):
    v = {}
    for w in words:
        try:
            i = featureDict[w]
        except KeyError:
            i = len(featureDict) + 1
            featureDict[w] = i
        try:
            v[i] += (1.0/len(words))
        except KeyError:
            v[i] = (1.0/len(words))
    return v

In [6]:
print toFeatureVector(preProcess(text))

{1: 0.125, 2: 0.125, 3: 0.125, 4: 0.125, 5: 0.125, 6: 0.125, 7: 0.125, 8: 0.125}


In [7]:
print featureDict

{'i': 2, 'am': 3, '.': 5, 'are': 6, 'you': 7, 'hello': 1, '?': 8, 'happy': 4}


In [8]:
# load training data from external files
trainData = []
trainLabels = []
with open('pos.csv', 'rb') as f:
    reader = unicodecsv.reader(f, encoding='utf-8')
    for line in reader:
        trainData.append(toFeatureVector(preProcess(line[0])))
        trainLabels.append(1.0)
with open('neg.csv', 'rb') as f:
    reader = unicodecsv.reader(f, encoding='utf-8')
    for line in reader:
        trainData.append(toFeatureVector(preProcess(line[0])))
        trainLabels.append(-1.0)
print len(featureDict)

7952


In [9]:
train_set = [(df, l) 
               for df in trainData
              for l in trainLabels]

In [10]:
# libraries for: NLP, machine learning
import nltk 

In [11]:
# train
model = nltk.NaiveBayesClassifier.train(train_set)

In [12]:
# predict
def analyseSentiment(text):
    v = toFeatureVector(preProcess(text))
    print v
    return model.classify(v)

In [13]:
s = analyseSentiment(text)
print "sentiment = ", s

{1: 0.125, 2: 0.125, 3: 0.125, 4: 0.125, 5: 0.125, 6: 0.125, 7: 0.125, 8: 0.125}
sentiment =  1.0


In [14]:
analyseSentiment("I am not happy")

{771: 0.25, 2: 0.25, 3: 0.25, 4: 0.25}


1.0